In [1]:
!wget https://opencorpora.org/files/export/dict/dict.opcorpora.xml.zip

--2025-10-08 11:18:50--  https://opencorpora.org/files/export/dict/dict.opcorpora.xml.zip
Resolving opencorpora.org (opencorpora.org)... 104.21.15.199, 172.67.163.210, 2606:4700:3030::6815:fc7, ...
Connecting to opencorpora.org (opencorpora.org)|104.21.15.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28757314 (27M) [application/zip]
Saving to: ‘dict.opcorpora.xml.zip’

dict.opcorpora.xml. 100%[===================>]  27.42M  --.-KB/s    in 0.1s    

2025-10-08 11:18:51 (201 MB/s) - ‘dict.opcorpora.xml.zip’ saved [28757314/28757314]



In [2]:
!unzip dict.opcorpora.xml.zip

Archive:  dict.opcorpora.xml.zip
  inflating: dict.opcorpora.xml      


## Зависимости и классы для n-грамм и лемматизатора

In [3]:
import xml.etree.ElementTree as ET
import re

In [4]:
import re
import os
import pickle
import math
from collections import Counter
from typing import List, Sequence, Callable, Optional, Iterable, Tuple

In [5]:
!pip install spacy
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 73.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [14]:
import spacy
spacy.load('ru_core_news_sm')

In [12]:
PAD = "<PAD>"

class POSNgramFreqBuilder:
    SENT_SPLIT_RE = re.compile(r'(?<=[.!?])\s+')
    WORD_RE = re.compile(r"\w+", flags=re.UNICODE)

    def __init__(self, n: int = 5):
        assert n >= 1 and n % 2 == 1, "n must be odd (1,3,5,...)"
        self.n = n
        self.k = n // 2
        self.counts = Counter()
        self.total = 0.0
        self._built = False

        # Внутри класса POSNgramFreqBuilder

    def get_pos_vocabulary(self) -> List[str]:
        """
        Возвращает список всех POS, которые встречаются в ключах counts (центральные позиции).
        Исключает PAD и, опционально, X.
        """
        vocab = set()
        for key in self.counts.keys():
            # key — tuple длины n
            if not key:
                continue
            # центральная позиция = k
            center = key[self.k]
            if center == PAD:
                continue
            vocab.add(center)
        # опция: убрать X, если он мешает
        if "X" in vocab:
            vocab.discard("X")
        return sorted(vocab)

    def choose_best_pos_for_unknown(self, left_pos_fixed: List[str], right_pos_fixed: Optional[List[str]] = None):
        """
        Специальная версия выбора POS, когда для слова нет кандидатов.
        Перебирает весь POS-вокабуляр и возвращает лучший (pos, score).
        left_pos_fixed: список уже выбранных POS слева (последний — ближе к центру)
        right_pos_fixed: по схеме greedy мы обычно оставляем пустым
        """
        import math
        if right_pos_fixed is None:
            right_pos_fixed = []
        best_pos = None
        best_score = -math.inf
        vocab = self.get_pos_vocabulary()
        # если vocab пуст — пробуем некоторый базовый список
        if not vocab:
            vocab = ["S","V","A","ADV","PR","CONJ","PART","NI","NUM","INTJ"]
        for cand in vocab:
            left_part = left_pos_fixed[-self.k:] if left_pos_fixed else []
            right_part = right_pos_fixed[:self.k] if right_pos_fixed else []
            window = list(left_part) + [cand] + list(right_part)
            if len(window) < self.n:
                window = window + [PAD] * (self.n - len(window))
            p = self.get_prob(window, smooth=True)
            score = math.log(p) if p > 0 else -math.inf
            if score > best_score:
                best_score = score
                best_pos = cand
        return best_pos, best_score

    # ----------------------------
    # Default mapping POS (UD -> target scheme)
    # Можно заменить при передаче map_pos аргумента
    # ----------------------------
    @staticmethod
    def map_pos_default(ud_pos):
        """
        Простейшее сопоставление меток Universal POS (spaCy/UDPipe) к твоим меткам:
        возвращает один из: S, NI, A, V, ADV, NUM, PR, CONJ, PART, INTJ, X
        """
        if not ud_pos:
            return "X"
        ud = ud_pos.upper()
        mapping = {
            "NOUN": "S", "PROPN": "S",
            "PRON": "NI",
            "ADJ": "A",
            "VERB": "V",
            "AUX": "V",
            "ADV": "ADV",
            "NUM": "NUM",
            "ADP": "PR",     # предлог
            "DET": "A",      # определитель — мэпим к прилагательному/артиклю (можно корректировать)
            "CCONJ": "CONJ",
            "SCONJ": "CONJ",
            "PART": "PART",
            "INTJ": "INTJ",
            "SYM": "X",
            "PUNCT": "X",
            "X": "X",
            "SPACE": "X"
        }
        return mapping.get(ud, "X")

    # ----------------------------
    # Вспомогательные: чтение и сегментация
    # ----------------------------
    @classmethod
    def _split_to_sentences(cls, text):
        parts = [p.strip() for p in cls.SENT_SPLIT_RE.split(text) if p.strip()]
        return parts

    @classmethod
    def _tokenize_words(cls, text: str) -> List[str]:
        return cls.WORD_RE.findall(text)

    # ----------------------------
    # Основной сборщик из уже тегированных предложений
    # tagged_sents: iterable of sequences of POS tags (target scheme)
    # ----------------------------
    def build_from_tagged(self, tagged_sents: Iterable[Sequence[str]], inplace: bool = True):
        counts = Counter()
        total = 0.0
        n = self.n
        k = self.k
        for sent in tagged_sents:
            L = len(sent)
            if L == 0:
                continue
            for i in range(L):
                left = max(0, i - k)
                right = min(L - 1, i + k)
                window = list(sent[left:right+1])
                if len(window) < n:
                    window = window + [PAD] * (n - len(window))
                counts[tuple(window)] += 1.0
                total += 1.0
        if inplace:
            self.counts = counts
            self.total = total
            self._built = True
        else:
            return counts, total

    # ----------------------------
    # Build from raw text with provided tagger function (per-sentence)
    # tagger_func(sentence_text) -> list of (word,pos) pairs OR a single list for that sentence
    # map_pos: function extern_pos -> target_pos
    # ----------------------------
    def build_from_text_with_tagger(self, text: str, tagger_func: Callable[[str], Sequence[Tuple[str, str]]],
                                    map_pos: Optional[Callable[[str], str]] = None):
        if map_pos is None:
            map_pos = self.map_pos_default
        sentences = self._split_to_sentences(text)
        tagged_sents = []
        for sent in sentences:
            try:
                tagged = tagger_func(sent)
            except Exception:
                # пропускаем предложение, если теггер упал на нём
                continue
            # теггер может возвращать формат: list of (word,pos) for this sentence,
            # или list of sentences (each list of pairs) — принимаем оба варианта
            if not tagged:
                continue
            # Detect nested output
            if isinstance(tagged[0], (list, tuple)) and len(tagged) > 0 and isinstance(tagged[0][0], (list, tuple)):
                # удалось вернуть список предложений
                for s in tagged:
                    pos_seq = [map_pos(pos) for (_w, pos) in s if pos is not None]
                    if pos_seq:
                        tagged_sents.append(pos_seq)
            else:
                # single sentence
                pos_seq = [map_pos(pos) for (_w, pos) in tagged if pos is not None]
                if pos_seq:
                    tagged_sents.append(pos_seq)
        # build counts
        self.build_from_tagged(tagged_sents, inplace=True)

    # из файлов бахаем
    def build_from_files(self, file_paths: List[str],
                         tagger: Optional[Callable[[str], Sequence[Tuple[str, str]]]] = None,
                         map_pos: Optional[Callable[[str], str]] = None,
                         encoding: str = 'utf-8',
                         chunk_sentences: int = 1000):
        """
        Читает файлы, разбивает на предложения, размечает с помощью tagger (по предложению),
        буферизует pos-последовательности и вызывает build_from_tagged пакетами.
        Если tagger is None — пробуем spaCy ru_core_news_sm автоматически.
        """
        if map_pos is None:
            map_pos = self.map_pos_default

        # если tagger не передан — пробуем spaCy
        if tagger is None:
            try:
                import spacy
                # попытаемся загрузить модель ru_core_news_sm, если не найдена — exception
                try:
                    nlp = spacy.load("ru_core_news_sm")
                except Exception as e:
                    # пробуем загрузить ru_core_news_sm; пользователь должен установить отдельно
                    raise RuntimeError("spaCy model 'ru_core_news_sm' не найдена. Установи модель: "
                                       "python -m spacy download ru_core_news_sm") from e

                def spacy_tagger(s: str):
                    doc = nlp(s)
                    # spaCy возвращает предложения — используем токены из doc.sents или весь doc
                    # Формируем список (word, pos) для каждого предложения в doc.sents
                    out = []
                    for sent in doc.sents:
                        sent_pairs = [(tok.text, tok.pos_) for tok in sent if tok.is_alpha or tok.pos_]
                        if sent_pairs:
                            out.append(sent_pairs)
                    # Если spaCy вернул несколько предложений, возвращаем их; иначе flatten
                    return out if out else []
                tagger = spacy_tagger
            except ImportError as e:
                raise RuntimeError("spaCy не установлена. Укажи tagger вручную или установи spaCy.") from e

        # Now iterate files, collect sentences, tag with tagger in streaming fashion
        buffer_pos_seqs = []
        buffered = 0
        for path in file_paths:
            if not os.path.exists(path):
                continue
            with open(path, "r", encoding=encoding, errors='ignore') as f:
                text = f.read()
            sentences = self._split_to_sentences(text)
            for s in sentences:
                # call tagger on sentence; it may return a single sentence or list of sentences
                try:
                    tagged = tagger(s)
                except Exception:
                    # если теггер падает на предложении — пропускаем
                    continue
                if not tagged:
                    continue
                # normalize returned formats
                if isinstance(tagged[0], (list, tuple)) and isinstance(tagged[0][0], (list, tuple)):
                    # list of sentences
                    for sent_pairs in tagged:
                        pos_seq = [map_pos(pos) for (_w, pos) in sent_pairs if pos is not None]
                        if pos_seq:
                            buffer_pos_seqs.append(pos_seq)
                            buffered += 1
                else:
                    # single sentence list of (w,pos)
                    pos_seq = [map_pos(pos) for (_w, pos) in tagged if pos is not None]
                    if pos_seq:
                        buffer_pos_seqs.append(pos_seq)
                        buffered += 1

                # If buffer full, feed to build_from_tagged batch and clear
                if buffered >= chunk_sentences:
                    # accumulate counts partial
                    # build_from_tagged with inplace=False returns counts tuple if we want; but easier —
                    # just call build_from_tagged on this batch and let it set internal counts,
                    # but we need to merge with existing counts; so use helper to merge
                    self._merge_from_tagged_batch(buffer_pos_seqs)
                    buffer_pos_seqs = []
                    buffered = 0

        # flush remaining
        if buffer_pos_seqs:
            self._merge_from_tagged_batch(buffer_pos_seqs)

        self._built = True

    # ----------------------------
    # Вспомогательный: merge batch of tagged sentences в self.counts
    # ----------------------------
    def _merge_from_tagged_batch(self, tagged_sents_batch: Iterable[Sequence[str]]):
        # считаем локальные counts и сливаем
        local_counts = Counter()
        local_total = 0.0
        n = self.n
        k = self.k
        for sent in tagged_sents_batch:
            L = len(sent)
            if L == 0:
                continue
            for i in range(L):
                left = max(0, i - k)
                right = min(L - 1, i + k)
                window = list(sent[left:right+1])
                if len(window) < n:
                    window = window + [PAD] * (n - len(window))
                local_counts[tuple(window)] += 1.0
                local_total += 1.0
        # merge into global
        for k, v in local_counts.items():
            self.counts[k] += v
        self.total += local_total

    def get_count(self, pos_sequence):
        if len(pos_sequence) != self.n:
            raise ValueError("pos_sequence must have length n")
        return float(self.counts.get(tuple(pos_sequence), 0.0))

    # находим вероятность для последовательности
    def get_prob(self, pos_sequence, smooth = True):
        if len(pos_sequence) != self.n:
            raise ValueError("pos_sequence must have length n")
        c = self.counts.get(tuple(pos_sequence), 0.0)
        if not smooth:
            return float(c) / float(self.total) if self.total > 0 else 0.0
        V = len(self.counts) or 1.0
        return float(c + 1.0) / float(self.total + V)

    def choose_best_pos_for_token(self, left_pos_fixed, right_pos_fixed, candidate_pos_list):
        import math
        best_pos = None
        best_score = -math.inf
        for cand in candidate_pos_list:
            left_part = left_pos_fixed[-self.k:] if left_pos_fixed else []
            right_part = right_pos_fixed[:self.k] if right_pos_fixed else []
            window = list(left_part) + [cand] + list(right_part)
            if len(window) < self.n:
                window = window + [PAD] * (self.n - len(window))
            p = self.get_prob(window, smooth=True)
            score = math.log(p) if p > 0 else -math.inf
            if score > best_score:
                best_score = score
                best_pos = cand
        return best_pos, best_score

    # save
    def save(self, path: str):
        with open(path, "wb") as f:
            pickle.dump({'n': self.n, 'counts': self.counts, 'total': self.total}, f)

    # load
    def load(self, path: str):
        with open(path, "rb") as f:
            data = pickle.load(f)
        self.n = data['n']
        self.k = self.n // 2
        self.counts = data['counts']
        self.total = data['total']
        self._built = True


In [13]:
class TextPreprocess:
    SENT_SPLIT_RE = re.compile(r'(?<=[.!?])\s+')

    def __init__(self, path_to_xml):
        opcorpa_tree = ET.parse(path_to_xml)
        opcorpa_tree_root = opcorpa_tree.getroot()

        self.gramm_dict = {
            "NOUN": "S",   "NPRO": "NI",
            "ADJF": "A",   "ADJS": "A",   "COMP": "A",   "ADJ": "A",
            "VERB": "V",   "INFN": "V",   "GRND": "V",   "PRTF": "V",   "PRTS": "V",
            "ADVB": "ADV", "ADV": "ADV",
            "NUMR": "NUM", "PREP": "PR",  "CONJ": "CONJ","PRCL": "PART","INTJ": "INTJ"
        }
        self.pos_gramm = self.gramm_dict.keys()

        self.wordform_to_lemma = {}
        self.create_detailed_dictionary_(opcorpa_tree_root)

    @staticmethod
    def normalize_word_(word):
        return word.strip().lower().replace('ё', 'е')

    # создаем словарь с леммами
    def create_detailed_dictionary_(self, xml_root):
        self.wordform_to_lemma = {}
        for lemma in xml_root.findall('.//lemma'):
            lemma_text = lemma.get('t')
            if lemma_text is None:
                head = lemma.find('l')
                if head is not None:
                    lemma_text = head.get('t')
                    tags = []
                    for g in head.findall('g'):
                        v = g.get('v')
                        if v:
                            tags.append(v.upper())
                    lemma_pos = "OTHER"
                    for t in tags:
                        if t in self.pos_gramm:
                            lemma_pos = self.gramm_dict.get(t, "OTHER")
                            break
                else:
                    lemma_text = None
            if not lemma_text:
                continue
            form_nodes = []
            form_nodes.extend(lemma.findall('l'))
            form_nodes.extend(lemma.findall('f'))
            seen = set()
            for form in form_nodes:
                wf = form.get('t')
                if not wf:
                    continue
                key = self.normalize_word_(wf)
                if (key, lemma_text) in seen:
                    continue
                seen.add((key, lemma_text))
                entry = {'lemma': lemma_text, 'pos': lemma_pos}
                if key in self.wordform_to_lemma.keys():
                    if entry not in self.wordform_to_lemma[key]:
                        self.wordform_to_lemma[key].append(entry)
                else:
                    self.wordform_to_lemma[key] = [entry]

    # удаляем пунктуацию и токенизируем
    @staticmethod
    def _delete_punctuation_marks(text):
        return re.sub(r'[^\w\s]', '', text)

    @staticmethod
    def _text_to_words(text):
        return [w for w in text.split() if w]

    # получаем все леммы для слова
    def _get_words_lemma(self, words: List[str]):
        results = []
        for word in words:
            word_lower = word.lower()
            normalize_word = self.normalize_word_(word)
            if normalize_word in self.wordform_to_lemma.keys():
                lemmas = self.wordform_to_lemma.get(normalize_word)
                result = {
                    "word": word,
                    "norm": normalize_word,
                    "lemmas": lemmas  # list of {'lemma':..., 'pos':...}
                }
            else:
                result = {
                    "word": word,
                    "norm": normalize_word,
                    "lemmas": []
                }
            results.append(result)
        return results

    # разбиение на предложения
    def split_to_sentences(self, text: str) -> List[str]:
        parts = [p.strip() for p in self.SENT_SPLIT_RE.split(text) if p.strip()]
        return parts

    # пайплайн для предложения + жадная дисамбигуация слева->направо
    def pipeline_sentence_greedy(self, sentence_text: str, builder: POSNgramFreqBuilder):
        """
        sentence_text: строка предложения (может содержать пунктуацию)
        builder: POSNgramFreqBuilder (с n заданным)
        """
        # Удаляем пунктуацию для лемматизации (как было в изначальном pipeline)
        text_wo_punct = self._delete_punctuation_marks(sentence_text)
        words = self._text_to_words(text_wo_punct)
        # получаем кандидаты для каждого слова
        lemmas_info = self._get_words_lemma(words)  # список dicts {'word','norm','lemmas'}
        L = len(lemmas_info)
        chosen_pos_seq = []   # уже выбранные POS (для левой части)
        result = []

        for i, tok in enumerate(lemmas_info):
            orig_word = tok['word']
            candidates = tok['lemmas']  # list of {'lemma','pos'}
            if not candidates:
                # пытаемся выбрать POS по соседям через билдер
                if builder is not None:
                    left_fixed = chosen_pos_seq[-builder.k:] if chosen_pos_seq else []
                    # greedy-left: не используем правую часть
                    chosen_pos, score = builder.choose_best_pos_for_unknown(left_fixed, right_pos_fixed=[])
                    if chosen_pos is None:
                        chosen_pos = 'X'
                else:
                    chosen_pos = 'X'
                # лемма по умолчанию — нормализованная форма; можно заменить pymorphy2 позже
                chosen_lemma = tok['norm']
                # (опционально) добавить в словарь как авторазбор:
                # self.wordform_to_lemma.setdefault(tok['norm'], []).append({'lemma': chosen_lemma, 'pos': chosen_pos})
                chosen_pos_seq.append(chosen_pos)
                result.append({
                    'word': orig_word,
                    'norm': tok['norm'],
                    'chosen_pos': chosen_pos,
                    'chosen_lemma': chosen_lemma,
                    'candidates': candidates,
                    'score': score if 'score' in locals() else None
                })
                continue

            # если однозначно — выбираем сразу
            if len(candidates) == 1:
                chosen_pos = candidates[0]['pos']
                chosen_lemma = candidates[0]['lemma']
                chosen_pos_seq.append(chosen_pos)
                result.append({
                    'word': orig_word,
                    'norm': tok['norm'],
                    'chosen_pos': chosen_pos,
                    'chosen_lemma': chosen_lemma,
                    'candidates': candidates
                })
                continue

            # неоднозначность — делаем жадный выбор опираясь на LEFT контекст
            # формируем список кандидатных POS
            candidate_pos_list = list({e['pos'] for e in candidates if e.get('pos')})
            # left_pos_fixed = последние k выбранных POS (если меньше — передаем короче)
            left_fixed = chosen_pos_seq[-builder.k:] if builder is not None else []
            right_fixed = []  # greedy-left: не используем правый контекст
            # выбираем лучший POS по builder
            if builder is None:
                # если билдера для разрешения омонимии нет
                # берем первую кандидатную пару
                chosen_pos = candidate_pos_list[0]
            else:
                chosen_pos, score = builder.choose_best_pos_for_token(left_fixed, right_fixed, candidate_pos_list)
                if chosen_pos is None:
                    # защита на случай -math.inf или проч.
                    chosen_pos = candidate_pos_list[0]
            # выбираем лемму, соответствующую chosen_pos (первая в списке)
            chosen_lemma = None
            for e in candidates:
                if e.get('pos') == chosen_pos:
                    chosen_lemma = e.get('lemma')
                    break
            if chosen_lemma is None:
                # если не нашли по POS — берем первую лемму
                chosen_lemma = candidates[0].get('lemma')

            chosen_pos_seq.append(chosen_pos)
            result.append({
                'word': orig_word,
                'norm': tok['norm'],
                'chosen_pos': chosen_pos,
                'chosen_lemma': chosen_lemma,
                'candidates': candidates
            })

        return result

    def pipeline_text_greedy(self, text: str, builder: POSNgramFreqBuilder):
        sentences = self.split_to_sentences(text)
        out = []
        for sent in sentences:
            sent_res = self.pipeline_sentence_greedy(sent, builder)
            out.append({'sentence': sent, 'tokens': sent_res})
        return out

## Пример использования

In [15]:
builder = POSNgramFreqBuilder(n=5)
builder.build_from_files(["voyna_i_mir.txt"])
builder.save("pos5_counts.pkl")

In [16]:
tp = TextPreprocess("dict.opcorpora.xml")

In [ ]:
text = "Все были рады. Он был очень рад, но не слишком долго."

In [ ]:
text = "Стала стабильнее экономическая и политическая обстановка, предприятия вывели из тени зарплаты сотрудников. Все Гришины одноклассники уже побывали за границей, он был чуть ли не единственным, кого не вывозили никуда дальше Красной Пахры."


In [17]:
text = "Я люблю русскую печь и печь пироги."

In [18]:
res = tp.pipeline_text_greedy(text, builder)
# res — список предложений, где для каждого токена есть chosen_pos и chosen_lemma
for sent in res:
    print("SENTENCE:", sent['sentence'])
    for tok in sent['tokens']:
        print(f"{tok['word']}({tok['chosen_lemma']}={tok['chosen_pos']})")

SENTENCE: Я люблю русскую печь и печь пироги.
Я(я=S)
люблю(люблю=V)
русскую(русский=A)
печь(печь=S)
и(и=S)
печь(печь=V)
пироги(пирог=S)


## Оценка точности

In [20]:
!wget https://opencorpora.org/files/export/annot/annot.opcorpora.xml.byfile.zip

--2025-10-08 11:46:59--  https://opencorpora.org/files/export/annot/annot.opcorpora.xml.byfile.zip
Resolving opencorpora.org (opencorpora.org)... 104.21.15.199, 172.67.163.210, 2606:4700:3031::ac43:a3d2, ...
Connecting to opencorpora.org (opencorpora.org)|104.21.15.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58733110 (56M) [application/zip]
Saving to: ‘annot.opcorpora.xml.byfile.zip’

annot.opcorpora.xml 100%[===================>]  56.01M  1.30MB/s    in 7.2s    

2025-10-08 11:47:06 (7.82 MB/s) - ‘annot.opcorpora.xml.byfile.zip’ saved [58733110/58733110]



In [21]:
!unzip annot.opcorpora.xml.byfile.zip

Archive:  annot.opcorpora.xml.byfile.zip
  inflating: 1000.xml                
  inflating: 1001.xml                
  inflating: 1002.xml                
  inflating: 1003.xml                
  inflating: 1004.xml                
  inflating: 1005.xml                
  inflating: 1006.xml                
  inflating: 1007.xml                
  inflating: 1008.xml                
  inflating: 1009.xml                
  inflating: 100.xml                 
  inflating: 1010.xml                
  inflating: 1011.xml                
  inflating: 1012.xml                
  inflating: 1013.xml                
  inflating: 1014.xml                
  inflating: 1015.xml                
  inflating: 1016.xml                
  inflating: 1017.xml                
  inflating: 1018.xml                
  inflating: 1019.xml                
  inflating: 101.xml                 
  inflating: 1020.xml                
  inflating: 1021.xml                
  inflating: 1022.xml                
  inflati

In [47]:
import xml.etree.ElementTree as ET
from collections import defaultdict
from sklearn.metrics import classification_report

def evaluate_on_opencorpora(xml_pathes, text_preproc, builder, max_tokens=10000):
    """
    xml_pathes: iterable of opencorpora XML files (annot.opcorpora-like)
    text_preproc: instance of TextPreprocess
    builder: instance of POSNgramFreqBuilder (используется в pipeline_sentence_greedy)
    max_tokens: лимит по числу токенов для сравнения
    """
    true_pos = []
    pred_pos = []

    for xml_path in xml_pathes:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        for sent in root.findall(".//sentence"):
            tokens = []
            gold_pos = []

            for token in sent.findall(".//token"):
                word = token.get("text")
                if not word:
                    continue
                l = token.find(".//l")
                if l is None:
                    continue
                g_tags = [g.get("v").upper() for g in l.findall(".//g") if g.get("v")]
                lemma_pos = None
                for tag in g_tags:
                    if tag in text_preproc.gramm_dict:
                        lemma_pos = text_preproc.gramm_dict[tag]
                        break
                if lemma_pos:
                    tokens.append(word)
                    gold_pos.append(lemma_pos)

            if not tokens:
                continue

            # === вот здесь важно: используем твой жадный пайплайн с билдером ===
            # pipeline_sentence_greedy ожидает предложение (строку) и внутри удаляет пунктуацию/нормализует
            # мы собираем "предложение" из токенов, чтобы поведение было детерминировано
            sent_text_for_tp = " ".join(tokens)
            tp_result = text_preproc.pipeline_sentence_greedy(sent_text_for_tp, builder)
            # tp_result — список словарей с 'norm' и 'chosen_pos'
            tp_tags = [t['chosen_pos'] for t in tp_result]

            # выровняем длины: tokens (gold) и tp_tags
            # Обычно lengths будут совпадать, но защитимся
            L = min(len(gold_pos), len(tp_tags))
            if L == 0:
                continue

            remaining = max_tokens - len(true_pos)
            if remaining <= 0:
                return _finalize(true_pos, pred_pos)

            # сколько можем добавить из текущего предложения
            add_n = min(L, remaining)
            true_pos.extend(gold_pos[:add_n])
            pred_pos.extend(tp_tags[:add_n])

            if len(true_pos) >= max_tokens:
                return _finalize(true_pos, pred_pos)

    return _finalize(true_pos, pred_pos)


def _finalize(true_pos, pred_pos):
    total = len(true_pos)
    acc = sum(t == p for t, p in zip(true_pos, pred_pos)) / total if total > 0 else 0.0
    print("Total compared:", total)
    print("Accuracy:", acc)
    print(classification_report(true_pos, pred_pos, digits=3))
    return {'total': total, 'accuracy': acc}


In [49]:
evaluate_on_opencorpora(["1015.xml", "1000.xml", "200.xml", "505.xml", "1.xml", "304.xml", "90.xml", "110.xml", "501.xml", "402.xml"], tp, builder)

Total compared: 4639
Accuracy: 0.781633972838974
              precision    recall  f1-score   support

           A      0.895     0.853     0.873       797
         ADV      0.578     0.773     0.661       207
        CONJ      0.897     0.213     0.344       451
        INTJ      0.000     0.000     0.000         6
          NI      0.667     0.710     0.688       107
         NUM      1.000     0.636     0.778        22
        PART      0.629     0.774     0.694       164
          PR      0.985     0.348     0.514       581
           S      0.727     0.985     0.836      1640
           V      0.890     0.986     0.936       664

    accuracy                          0.782      4639
   macro avg      0.727     0.628     0.632      4639
weighted avg      0.817     0.782     0.751      4639



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'total': 4639, 'accuracy': 0.781633972838974}

In [50]:
import xml.etree.ElementTree as ET
from collections import Counter
from sklearn.metrics import classification_report

def evaluate_simple_on_opencorpora(xml_pathes, text_preproc, max_tokens=10000):
    """
    xml_pathes: iterable of xml files (annot.opcorpora-like)
    text_preproc: instance of TextPreprocess
    max_tokens: limit on number of token-level comparisons (words)
    """
    true_pos = []
    pred_pos = []

    for xml_path in xml_pathes:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        for sent in root.findall(".//sentence"):
            tokens = []
            gold_pos = []

            for token in sent.findall(".//token"):
                word = token.get("text")
                # защитно: пропускаем если нет текста
                if not word:
                    continue
                l = token.find(".//l")
                if l is None:
                    continue
                g_tags = [g.get("v").upper() for g in l.findall(".//g") if g.get("v")]
                lemma_pos = None
                for tag in g_tags:
                    if tag in text_preproc.gramm_dict:
                        lemma_pos = text_preproc.gramm_dict[tag]
                        break
                if lemma_pos:
                    tokens.append(word)
                    gold_pos.append(lemma_pos)

            if not tokens:
                # пустое предложение — пропускаем
                continue

            # получаем предсказания
            preds = text_preproc._get_words_lemma(tokens)
            pred_tags = []
            for p in preds:
                if not p["lemmas"]:
                    pred_tags.append("X")
                elif len(p["lemmas"]) == 1:
                    pred_tags.append(p["lemmas"][0]["pos"])
                else:
                    pred_tags.append(p["lemmas"][0]["pos"])

            # если добавление этого предложения превысит лимит, обрезаем
            remaining = max_tokens - len(true_pos)
            if remaining <= 0:
                # достигли лимита — выйти полностью
                return _finalize(true_pos, pred_pos)

            if len(gold_pos) <= remaining:
                true_pos.extend(gold_pos)
                pred_pos.extend(pred_tags)
            else:
                true_pos.extend(gold_pos[:remaining])
                pred_pos.extend(pred_tags[:remaining])
                return _finalize(true_pos, pred_pos)

    return _finalize(true_pos, pred_pos)


def _finalize(true_pos, pred_pos):
    total = len(true_pos)
    acc = sum(t == p for t, p in zip(true_pos, pred_pos)) / total if total > 0 else 0.0
    print("Total compared:", total)
    print("Accuracy:", acc)
    print(classification_report(true_pos, pred_pos, digits=3))
    return {'total': total, 'accuracy': acc}


In [51]:
evaluate_simple_on_opencorpora(["1015.xml", "1000.xml", "200.xml", "505.xml", "1.xml", "304.xml", "90.xml", "110.xml", "501.xml", "402.xml"], tp)

Total compared: 4639
Accuracy: 0.9495580944169002
              precision    recall  f1-score   support

           A      0.966     0.900     0.932       797
         ADV      0.912     0.952     0.931       207
        CONJ      0.961     0.984     0.973       451
        INTJ      1.000     1.000     1.000         6
          NI      0.832     0.785     0.808       107
         NUM      0.929     0.591     0.722        22
        PART      0.881     0.945     0.912       164
          PR      1.000     0.917     0.957       581
           S      0.955     0.990     0.972      1640
           V      0.917     0.952     0.934       664

    accuracy                          0.950      4639
   macro avg      0.935     0.902     0.914      4639
weighted avg      0.950     0.950     0.949      4639



{'total': 4639, 'accuracy': 0.9495580944169002}